In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  #w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = re.sub(r"[0-9]"," ",w)
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [ ]:
!wget --header="Host: doc-0s-0c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://drive.google.com/drive/u/1/folders/1cVuZ2bjX0R8-Pr3Vr0ZsGOV0Rx4tL3Vy" --header="Cookie: AUTH_oq0eqiav6qgerbf19mn4aj6aebbh3v3f_nonce=7bh210qdhp7jm; _ga=GA1.2.1218104909.1587743248" --header="Connection: keep-alive" "https://doc-0s-0c-docs.googleusercontent.com/docs/securesc/spvaadfbrokq707dv7sq6kugikklrfu9/n8kc94kvc7ldc0mjivff60lk7egfr0if/1598340150000/12952873562170756528/12952873562170756528/1COeaLwfspYMvX2dntnr5treoGLR8mS3R?e=download&authuser=1&nonce=7bh210qdhp7jm&user=12952873562170756528&hash=tvtifq1dran5l4fq44olkefappj4p5an" -c -O 'ita.txt'

--2020-08-25 07:23:00--  https://doc-0s-0c-docs.googleusercontent.com/docs/securesc/spvaadfbrokq707dv7sq6kugikklrfu9/n8kc94kvc7ldc0mjivff60lk7egfr0if/1598340150000/12952873562170756528/12952873562170756528/1COeaLwfspYMvX2dntnr5treoGLR8mS3R?e=download&authuser=1&nonce=7bh210qdhp7jm&user=12952873562170756528&hash=tvtifq1dran5l4fq44olkefappj4p5an
Resolving doc-0s-0c-docs.googleusercontent.com (doc-0s-0c-docs.googleusercontent.com)... 64.233.184.132, 2a00:1450:400c:c0b::84
Connecting to doc-0s-0c-docs.googleusercontent.com (doc-0s-0c-docs.googleusercontent.com)|64.233.184.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘ita.txt’

ita.txt                 [  <=>               ]  46.25M   124MB/s    in 0.4s    

2020-08-25 07:23:01 (124 MB/s) - ‘ita.txt’ saved [48492727]



In [ ]:
import pandas as pd
df = pd.read_csv("ita.txt", delimiter='\t', names=list(range(3)))
df = df.iloc[:,0:2]
df.columns = ['English', 'Italian']
df = df[df.columns[::-1]]
#data = df.sample(n=200000)

data_ita = df['Italian']
data_eng = df['English']

#changed to English to italic, hence not chnaging any variable
#data_eng = df['Bengali']
#data_ita = df['English']

In [ ]:
df.shape

(340432, 2)

In [ ]:
preprocessed_ita = [preprocess_sentence(data) for data in data_ita]
preprocessed_eng = [preprocess_sentence(data) for data in data_eng]

In [ ]:
input_tensor, inp_lang = tokenize(preprocessed_ita)
target_tensor, targ_lang = tokenize(preprocessed_eng)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [ ]:
max_length_targ 

112

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.25)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

255324 255324 85108 85108


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 100
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_tar_size


13615

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 102]), TensorShape([64, 112]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 102, 256)
Encoder Hidden state shape: (batch size, units) (64, 256)


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 256)
Attention weights shape: (batch_size, sequence_length, 1) (64, 102, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 13615)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './test_chk1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 6

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 1000 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.6224
Epoch 1 Batch 1000 Loss 0.2254
Epoch 1 Batch 2000 Loss 0.1515
Epoch 1 Batch 3000 Loss 0.1243
Epoch 1 Loss 0.1860
Time taken for 1 epoch 3934.36234998703 sec

Epoch 2 Batch 0 Loss 0.0867
Epoch 2 Batch 1000 Loss 0.0826
Epoch 2 Batch 2000 Loss 0.0614
Epoch 2 Batch 3000 Loss 0.0790
Epoch 2 Loss 0.0777
Time taken for 1 epoch 3772.265832424164 sec

Epoch 3 Batch 0 Loss 0.0507
Epoch 3 Batch 1000 Loss 0.0508
Epoch 3 Batch 2000 Loss 0.0513
Epoch 3 Batch 3000 Loss 0.0356
Epoch 3 Loss 0.0476
Time taken for 1 epoch 3778.4082736968994 sec

Epoch 4 Batch 0 Loss 0.0398
Epoch 4 Batch 1000 Loss 0.0295
Epoch 4 Batch 2000 Loss 0.0260
Epoch 4 Batch 3000 Loss 0.0341
Epoch 4 Loss 0.0350
Time taken for 1 epoch 3775.546888113022 sec

Epoch 5 Batch 0 Loss 0.0204
Epoch 5 Batch 1000 Loss 0.0255
Epoch 5 Batch 2000 Loss 0.0229
Epoch 5 Batch 3000 Loss 0.0343
Epoch 5 Loss 0.0276
Time taken for 1 epoch 3784.1292927265167 sec

Epoch 6 Batch 0 Loss 0.0236
Epoch 6 Batch 1000 Loss 0.0355
Epoch

In [ ]:
max_length_inp

102

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)
  
  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  sentence = sentence.replace('<start>', '')
  sentence = sentence.replace('<end>', '')
  result = result.replace('<start>', '')
  result = result.replace('<end>', '')
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  #plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
df.tail

<bound method NDFrame.tail of                                                   Italian                                            English
0                                                   Ciao!                                                Hi.
1                                                  Corri!                                               Run!
2                                                  Corra!                                               Run!
3                                                Correte!                                               Run!
4                                                    Chi?                                               Who?
...                                                   ...                                                ...
340427  Se vuoi sembrare un madrelingua, devi essere d...  If you want to sound like a native speaker, yo...
340428  Se vuoi sembrare un madrelingua, devi essere d...  If you want to sound like a native spea

In [ ]:
translate(u'lui se ne sta andando')

Input:  lui se ne sta andando 
Predicted translation: he is leaving .  


In [ ]:
import pickle
from google.colab import files

pickle.dump((inp_lang, targ_lang, max_length_targ,max_length_inp, vocab_inp_size, vocab_tar_size, BUFFER_SIZE, steps_per_epoch),open('all_data.pkl','wb'))
files.download('all_data.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r checkpoint.zip 'test_chk1'
#Download files
#files.download('checkpoint.zip')

  adding: test_chk1/ (stored 0%)
  adding: test_chk1/checkpoint (deflated 38%)
  adding: test_chk1/ckpt-3.index (deflated 69%)
  adding: test_chk1/ckpt-1.index (deflated 69%)
  adding: test_chk1/ckpt-3.data-00000-of-00001 (deflated 15%)
  adding: test_chk1/ckpt-2.data-00000-of-00001 (deflated 15%)
  adding: test_chk1/ckpt-1.data-00000-of-00001 (deflated 15%)
  adding: test_chk1/ckpt-2.index (deflated 69%)


In [ ]:
import dill as dill
dill.dump((encoder, decoder),open('encoder_decoder.pkl','wb'))